In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import datetime as dt

In [ ]:
df = pd.read_table('python_scripts/ETL_python_scripts/content.txt', sep="|", header=0, index_col=1)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df = df.dropna(axis=1, how='all')

In [ ]:
df = df.drop([':------------------------------------------------------'])

In [ ]:
df.columns

In [ ]:
df.columns = ['date', 'time', 'city_name', 'temp_c', 'temp_f']

In [ ]:
df.head()

In [ ]:
df.info()

In [3]:
import sys

In [4]:
sys.path.append('python_scripts/ETL_python_scripts/')

# write to DB

In [5]:
import ETL_secrets

In [6]:
import db_connection

In [7]:
#importing public libraries
from sqlalchemy import create_engine


In [8]:
engine = db_connection.engine

In [10]:
table_name = 'weather'

In [ ]:
df.to_sql(table_name, engine, if_exists='replace')

# read from DB

In [11]:
sql_query = f'select * from {table_name}'
df_from_db = pd.read_sql_query(sql_query, con=engine)

In [12]:
df_from_db

,weather_id,date,time,city_name,temp_c,temp_f
0,202301-2808-2155-27638a26-1510-408c-a198-954c...,2023-01-28,08:21:55.778829,Moscow,-3,26
1,202301-2808-2158-a526693d-ba05-4d06-b45c-d8a4...,2023-01-28,08:21:58.485620,London,4,39


удивительно, но пишет в БД и даже вычитывает.
Надо разобраться с пересборой контейнеров. Почему-то опять были проблемы с библиотекой tabulate хотя в коде ее нет.